In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 3], ['d', 4]],
                   columns=['letter', 'number'])

In [3]:
df1

,letter,number
0,a,1
1,b,2


In [4]:
df2

,letter,number
0,c,3
1,d,4


In [6]:
pd.concat([df1,df2])

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [7]:
a = [1,2,3,4,5,6,7,8]

In [12]:
a[0:8:2]

[1, 3, 5, 7]

In [17]:
b = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g'])
b

array(['a', 'b', 'c', 'd', 'e', 'f', 'g'], dtype='<U1')

In [19]:
np.array_split(b, 2)

[array(['a', 'b', 'c', 'd'], dtype='<U1'), array(['e', 'f', 'g'], dtype='<U1')]

In [ ]:
b.reshape((2,))